In [1]:
import random

# VNS - variable neighbourhood search
# i ova metoda pokusava da promeni nedostatak lokalne pretrage ali na drugaciji nacin

# u lokalnoj pretrazi biramo novo potencijalno resenje iz neke male okoline trenutnog resenja(mala izmena)
# ovde postepeno prosirujemo okolone tog potecnijalnog resenja dok ne nadjemo nesto bolje

In [5]:
# koristimo iste pomocne funkcije id proslog primera
def isFeasible(solution):
    for resource in solution:
        if resource:
            return True
    return False;


def initialize(numResources):
    solution = [random.random() < 0.25 for _ in range(numResources)]
    if not isFeasible(solution):
        chosenResource = random.randRange(0, numResources)
        solution[chosenResource] = True
    return solution


def calcSolutionValue(solution, cost, fixedCost):
    value = 0
    numUsers = len(cost)
    numResources = len(fixedCost)
    
    used = [False for _ in range(numResources)]
    
    for i in range(numUsers):
        minCost = float('inf')
        resourceUsed = -1
        for j in range(numResources):
            if solution[j] and cost[i][j] < minCost:  
                minCost = cost[i][j]
                resourceUsed = j   
        value += minCost
        used[resourceUsed] = True
        
    for i in range(numResources):
        if used[i]:
            value += fixedCost[i]
    
    solution = used
    return value



def invert(solution):
    chouseResource = random.randrange(0, len(solution))
    solution[chosenResource] = not solution[chosenResource]
    if isFeasible(solution):
        return chosenResource
    else:  
        solution[chosenResource] = not solution[chosenResource]
        return -1

In [7]:
# pomocna funkcija za VNS
# u nasem primeru, k-ta okolina znaciti da dozvolimo da se invertuje k 
# vrednosti u resenju

def getNeighbor(solution, k):
# ovo u komentaru je lose resenje
# problem sa ovim je sto moze da se desi ekstremni slucaj da se svaki put 
# invertuje isti bit i zato radimo sa sample
#     for i in range(k):
#         chooseIndex = random.randrange(0, len(solution))
#         solution[chosenIndex] = not solution[chosenIndex]
    indices = range(len(solution))
    to_invert = random.sample(indices, k) 
    for index in toInvert:
        solution[index] = not solution[index]
        
    # obavezno proverimo da li je dobijeno resenje uopste dopustivo
    if isFeasible(solution):
        return True, toInvert
    else:
        # NOTE: na vezbama je vracao ovo toInvert pa onda radio restore u glavnoj funkciji
        # ali mzoe samo i ovde umseto komentara da se invertuje opet i to je lepse
        return False, toInvert

In [8]:
def restore(solution, invertedIndices):
    for resource in invertedIndices:
        solution[resource] = not solution[resource]

In [11]:
# VNS ima dosta varijacija, mi ovde radimo RVNS

def RVNS(cost, fixedCost, iters, max_k):
    solution = initialize(len(fixedCost))
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue

    for i in range(iters):
        k = 0
        while k <= max_k:       # testiramo okolone do kaksimalnog dozvoljenog broja
            success, invertedIndices = getNeighbor(solution, k)   # neki random sused u k-toj okoloni
            if not success:
                restore(solution, invertedIndices)
                continue
            newValue = calcSolutionValue(solution, cost, fixedCost)
            if newValue < currValue:
                currValue = newValue
                if newValue < bestValue:
                    bestValue = newValue
                break   # mozemo da izadjemo iz unutrasnje petlje iako nismo dosli do max_k ako smo nasli bolje
            else:
                k += 1
                restore(solution, invertedIndices)
                

    return bestValue


In [12]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readLine().split()]
        cost = [[int(x) for x in f.readLine().split()] for _ in range(numUsers)]
        fixedCost = [int(x) for x in f.readLine.split()]
        return cost, fixedCost
    
cost, fixedCost = readInput('uflp1.txt')

# pazimo samo kada biramo max_k da bude validno, npr ovde bi za 4 pukao program zbog velisine
# samog resenja, a i 3 nam je nekako previse
RVNS(cost, fixedCost, 10, 2)

FileNotFoundError: [Errno 2] No such file or directory: 'uflp1.txt'